In [31]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
%matplotlib inline

In [2]:
conn = psycopg2.connect('dbname = mh')

In [3]:
cur = conn.cursor()

In [4]:
query_post = """
    Select * from pc_post;
    """

query_user = """
    Select * from pc_user;
    """

In [15]:
cur.execute(query_post)

In [16]:
post = cur.fetchall()

### Text Cleaning and getting into document format

In [32]:
post_code = []
users = []
documents = []
type_post = []


for info in post:
    post_code.append(info[0])
    users.append(info[1])
    content = [info[2]]
    type_post.append(info[3])
    cleaned = []
    for line in content:
        new_string = "".join([letter for letter in line if letter in string.printable]).strip()
        cleaned.append(" ".join(new_string.split()))
    documents.append(" ".join(cleaned))

### Creating pandas df of post info

In [33]:
post_df = pd.DataFrame({'post_code': post_code, 'users': users, 'documents': documents, 'type_post': type_post})

In [36]:
post_df

,documents,post_code,type_post,users
0,Default Management and Coping Skills for Anxie...,5986256,author,Gus1234U
1,"this is a good thread, and would probably beni...",5986345,responder,shattered sanity
2,Do you find that your responses to social inte...,5985119,author,BBB2
3,I'm horrible at interacting with others in per...,5985413,responder,sonjaward809
4,I am pretty good. Not necessarily comfortable ...,5986382,responder,justafriend306
5,I feel like I'm either too sharp or too loud w...,5986392,responder,Fharraige
6,I am not sure where it comes from and only thi...,5979749,author,It'sjustmehere
7,"I tend to use a script when talking to people,...",5986424,responder,Nike007
8,I would discuss it with the doc. Maybe the med...,5979750,responder,TishaBuv
9,Quote: Originally Posted by Fharraige I feel l...,5986729,responder,Unhinged88


# NLP

## Text Processing Pipeline

In [37]:
from nltk.corpus import stopwords

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [40]:
tfidf_vect = TfidfVectorizer(stop_words = 'english')

In [41]:
tfidf_vectorized = tfidf_vect.fit_transform(documents)

In [42]:
from sklearn.metrics.pairwise import linear_kernel

In [45]:
len(tfidf_vect.vocabulary_)

8533

In [46]:
cos_sim = linear_kernel(tfidf_vectorized, tfidf_vectorized)